In [1]:
!pip install openai gradio python-dotenv PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 59.9 MB/s eta 0:00:00


In [2]:
import os
from dotenv import load_dotenv

# If you have a .env file with: OPENAI_API_KEY=your_key_here
load_dotenv()

# OR directly set the key (temporary for testing)
# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxx"

# Check if the key is loaded correctly
if "OPENAI_API_KEY" in os.environ:
    print(" OpenAI API key loaded.")
else:
    print(" API key not found. Please set it.")


✅ OpenAI API key loaded.


In [8]:
import fitz  # PyMuPDF

# Correct paths based on your Colab file tree
txt_path = "/business_summary.txt"
pdf_path = "/about_business.pdf"

def load_txt(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def load_pdf(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

try:
    business_text = load_txt(txt_path)
    business_pdf_text = load_pdf(pdf_path)
    knowledge_base = business_text + "\n\n" + business_pdf_text
    print(" Knowledge base loaded successfully.")
    print(f"Characters loaded: {len(knowledge_base)}")
except Exception as e:
    print(" Error loading files:", e)


✅ Knowledge base loaded successfully.
Characters loaded: 4400


In [9]:
# Step 2 - Tool functions

def record_customer_interest(email: str, name: str, message: str):
    """
    Logs a potential customer lead. In this assignment, printing is enough.
    """
    log = f"[LEAD] Name: {name}, Email: {email}, Message: {message}"
    print(log)
    # Optional: save to a file
    with open("customer_leads.log", "a", encoding="utf-8") as f:
        f.write(log + "\n")

def record_feedback(question: str):
    """
    Logs questions the chatbot couldn't answer.
    """
    log = f"[FEEDBACK] Unanswered question: {question}"
    print(log)
    # Optional: save to a file
    with open("feedback.log", "a", encoding="utf-8") as f:
        f.write(log + "\n")

# Test the functions
record_customer_interest("test@example.com", "John Doe", "Interested in sourdough delivery.")
record_feedback("Do you offer gluten-free croissants?")


[LEAD] Name: John Doe, Email: test@example.com, Message: Interested in sourdough delivery.
[FEEDBACK] Unanswered question: Do you offer gluten-free croissants?


In [10]:
# Step 3 - System Prompt & Chat Setup

from openai import OpenAI

client = OpenAI()

system_prompt = f"""
You are hamdan_bakery, a friendly neighborhood artisan bakery chatbot 🍞.
Use ONLY the information provided below to answer user questions.
Do not make up or invent details — if you don't find the answer,
call the record_feedback tool.

Knowledge base:
{knowledge_base}

Guidelines:
- Speak in a warm and concise tone, as if you are the bakery.
- When users ask about orders, products, or services, refer to the text above.
- Encourage users to leave their name and email for pre-orders, custom cakes, or delivery.
- If a question is outside your knowledge, log it with record_feedback(question).
"""

print("System prompt set.")


✅ System prompt set.


In [16]:
# Step 4 - Agent interaction loop using OpenAI API

import json

# Define the tool schemas for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": "Record customer leads with name, email, and message",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {"type": "string"},
                    "name": {"type": "string"},
                    "message": {"type": "string"}
                },
                "required": ["email", "name", "message"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Log a question the bot cannot answer",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string"}
                },
                "required": ["question"]
            },
        },
    }
]

def run_agent(user_input, chat_history):
    """
    Handles user input, sends it to OpenAI, executes tools if needed, and returns the assistant's response.
    """
    messages = [
        {"role": "system", "content": system_prompt}
    ] + chat_history + [{"role": "user", "content": user_input}]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    message = response.choices[0].message

    # Check if the model requested a tool call
    if message.tool_calls:
        for tool_call in message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)

            if tool_name == "record_customer_interest":
                record_customer_interest(**tool_args)
                return "Thanks for your interest! We’ve logged your info and will get back to you soon 📝"

            elif tool_name == "record_feedback":
                record_feedback(**tool_args)
                return "Hmm, I’m not sure about that — but I’ve logged your question so we can follow up 👌"

    # Otherwise, return the assistant's text
    return message.content

#  Quick test in notebook
chat_history = []
user_input = "Hi, who is the president of Algeria?"
assistant_reply = run_agent(user_input, chat_history)
chat_history.append({"role": "user", "content": user_input})
chat_history.append({"role": "assistant", "content": assistant_reply})

print(" Bot:", assistant_reply)


[FEEDBACK] Unanswered question: Who is the president of Algeria?
🤖 Bot: Hmm, I’m not sure about that — but I’ve logged your question so we can follow up 👌


In [17]:
import gradio as gr
gr.close_all()  # closes any previously running Gradio instances


Closing server running on port: 7860


In [19]:
import traceback
import gradio as gr

chat_history = []

def chatbot_interface(user_input, history):
    global chat_history
    try:
        assistant_reply = run_agent(user_input, chat_history)
        chat_history.append({"role": "user", "content": user_input})
        chat_history.append({"role": "assistant", "content": assistant_reply})
        return assistant_reply
    except Exception as e:
        print(" Error in chatbot:", e)
        import traceback
        traceback.print_exc()
        return f" Internal error: {e}"

demo = gr.ChatInterface(
    fn=chatbot_interface,
    title="🥐 hamdan_bakery Chatbot",
    description="Ask me about our breads, pastries, cakes, or place an order!",
)

demo.launch(share=True, debug=True)



/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0ce1b43556da855991.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[FEEDBACK] Unanswered question: Who is the CEO of hamdan_bakery?
[FEEDBACK] Unanswered question: who is the owner of hamdan_bakery?
[FEEDBACK] Unanswered question: who is the president of syria ?
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0ce1b43556da855991.gradio.live
